In [1]:
# Prior to running the code, make sure to download sample_<year>.zip from the Freddie Mac website and unzip them and store them all in the same folder. 
# Provide below the name for the folder where sample_<year> folder are all stored. The output csv file will be copied in this folder. 
 
folder_path = '/Users/yuyaogawa/Documents/Home Work/portfolio/loan_forecast/fraddie_mae_data'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
performance_data = pd.read_csv('loan_performance.csv')

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/225001025.py:1: DtypeWarning: Columns (26,27,28,29,32,35,39,55,56,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  performance_data = pd.read_csv('loan_performance.csv')


In [22]:
print('Below shows the count of each status')
print(performance_data['ZERO_BALANCE_CODE'].value_counts())
print('---------------------------------')
print('total number of unique loan is', len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()))
print('there is about this many missing zero balance status:',
      len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()) - sum(performance_data['ZERO_BALANCE_CODE'].value_counts()[1:]))

Below shows the count of each status
ZERO_BALANCE_CODE
1.0     72880
9.0       931
16.0      433
3.0       371
96.0      274
2.0       252
15.0      102
Name: count, dtype: int64
---------------------------------
total number of unique loan is 96000
there is about this many missing zero balance status: 20757


/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/3047140243.py:6: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()) - sum(performance_data['ZERO_BALANCE_CODE'].value_counts()[1:]))


In [23]:
# If Zero balance code is 1 (prepaid or matured) and the date the balance became zero is before the maturity, the loan is fully prepaid.
condition1 = (performance_data['ZERO_BALANCE_CODE'] ==  1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] < performance_data['MATURITY_DATE'])
# If zero balance code is 1 and zero balance effective date is the same day as maturity, then loan must be matured. 
condition2 = (performance_data['ZERO_BALANCE_CODE'] == 1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] == performance_data['MATURITY_DATE'])
# If zero balance code is 3, 2, 96, or 9, it is a default. 
condition3 = performance_data['ZERO_BALANCE_CODE'].isin([9, 96, 3, 2])
# If zero balance code is none of above, we call loan is censored. 
condition4 = performance_data['ZERO_BALANCE_CODE'].isin([15,16])

conditions = [condition1, condition2, condition3, condition4]

values = ['Full Prepayment', 'Matured', 'Default', 'Censored']

# Create the new column using np.select
performance_data['Event_status'] = np.select(conditions, values)
performance_data['Prepayment_status'] = np.select([(performance_data['Event_status']=='Full Prepayment')], [1])
performance_data['Default_status'] = np.select([(performance_data['Event_status'] == 'Default')], [1])

In [24]:
print('Below shows the count of each status')
print(performance_data['Event_status'].value_counts())
print(performance_data['Prepayment_status'].value_counts())
print(performance_data['Default_status'].value_counts())

Below shows the count of each status
Event_status
0                  5163224
Full Prepayment      72705
Default               1828
Censored               535
Matured                158
Name: count, dtype: int64
Prepayment_status
0    5165745
1      72705
Name: count, dtype: int64
Default_status
0    5236622
1       1828
Name: count, dtype: int64


In [25]:
# Create a start and stop time for survival analysis
performance_data['Start_time'] = performance_data['LOAN_AGE']
performance_data['Stop_time'] = performance_data['LOAN_AGE'] + 1

In [26]:
print(min(performance_data['Start_time'].unique()))
print(max(performance_data['Start_time'].unique()))

0
298


In [27]:
# Stratify the loan term
performance_data['ORIGINAL_LOAN_TERM'].value_counts()

ORIGINAL_LOAN_TERM
360    3561290
180    1183419
240     311696
120      89099
300      41655
        ...   
268          8
287          7
246          5
181          4
151          2
Name: count, Length: 154, dtype: int64

In [28]:
condition1 = performance_data['ORIGINAL_LOAN_TERM'].isin(range(0,240))
condition2 = performance_data['ORIGINAL_LOAN_TERM'].isin(range(240,301))
condition3 = performance_data['ORIGINAL_LOAN_TERM'] > 300

conditions = [condition1, condition2, condition3]

values = ['0-19yr', '20-25yr', '+25yr']

performance_data['TERM'] = np.select(conditions, values)

In [29]:
performance_data['TERM'].value_counts()

TERM
+25yr      3581443
0-19yr     1293261
20-25yr     363746
Name: count, dtype: int64

In [30]:
performance_data['CREDIT_SCORE'].replace(9999, np.nan, inplace=True)

# Define the bins and labels for categorical scores
bins = [300, 580, 670, 740, 800, 850]
labels = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']

# Create a new column 'credit_score_category' with the categorical scores
performance_data['SCORE'] = pd.cut(performance_data['CREDIT_SCORE'], bins=bins, labels=labels, right=False, include_lowest=True)

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/478400689.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  performance_data['CREDIT_SCORE'].replace(9999, np.nan, inplace=True)


In [31]:
performance_data['SCORE'].value_counts()

SCORE
Very Good    2370025
Good         1676929
Fair          608956
Excellent     544234
Poor           31390
Name: count, dtype: int64

In [32]:
bins = [30000, 60000, 90000, 120000, 150000, 180000, 210000, 240000, 270000, 300000, 500000, 1000000]
labels = ['3-60K', '6-90K', '9-120K', '12-150K', '15-180K', '18-210K', '21-240K', '24-270K', '27-300K', '3-500K', '500k-1M']

# Create a new column 'credit_score_category' with the categorical scores
performance_data['SIZE'] = pd.cut(performance_data['ORIGINAL_UPB'], bins=bins, labels=labels, right=False, include_lowest=True)

In [33]:
performance_data['SIZE'].value_counts()

SIZE
9-120K     761878
12-150K    724263
3-500K     678926
6-90K      664696
15-180K    587449
18-210K    496734
21-240K    378545
3-60K      318147
24-270K    303339
27-300K    229180
500k-1M     72508
Name: count, dtype: int64

In [34]:
performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)

bins = [20, 40, 60, 80, 100, 120, 150, 200, 500]
labels = ['20-40%', '40-60%', '60-80%', '80-100%', '100-120%', '120-150%', '150-200%', '200-500%']

performance_data['LTV'] = pd.cut(performance_data['ORIGINAL_LOAN-TO-VALUE'], bins=bins, labels=labels, right=False, include_lowest=True)

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/1192586942.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)


In [35]:
performance_data['LTV'].value_counts()

LTV
80-100%     2045083
60-80%      1808207
40-60%       874597
20-40%       335949
100-120%      90395
120-150%      30752
150-200%      11511
200-500%       3060
Name: count, dtype: int64

In [36]:
print(performance_data['AMORTIZATION_TYPE'].value_counts())
# All of the loans are fixed rate.

AMORTIZATION_TYPE
FRM    5238450
Name: count, dtype: int64


In [37]:
# calculate the average of current interest rate for each month in a given year. 
monthly_avg_interest_rate = performance_data.groupby(['MONTHLY_REPORTING_PERIOD','SCORE'])['CURRENT_INTEREST_RATE'].mean().reset_index()
monthly_avg_interest_rate['CURRENT_INTEREST_RATE'].fillna(performance_data['CURRENT_INTEREST_RATE'].mean(),inplace=True)
monthly_avg_interest_rate.rename(columns={'CURRENT_INTEREST_RATE': 'Avg_Curr_Interest_Rate'}, inplace=True)


performance_data = pd.merge(performance_data, monthly_avg_interest_rate, on = ['MONTHLY_REPORTING_PERIOD', 'SCORE'], how = 'left')
performance_data['ORIGINAL_INTEREST_RATE'].fillna(performance_data['ORIGINAL_INTEREST_RATE'].mean(),inplace=True)

performance_data['Rate_gap'] = performance_data['Avg_Curr_Interest_Rate'] - performance_data['ORIGINAL_INTEREST_RATE']

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/3867774684.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_avg_interest_rate = performance_data.groupby(['MONTHLY_REPORTING_PERIOD','SCORE'])['CURRENT_INTEREST_RATE'].mean().reset_index()
/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/3867774684.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perfo

In [38]:
print(sum(performance_data['ORIGINAL_INTEREST_RATE'].isna()))
print(sum(performance_data['Rate_gap'].isna()))

0
6916


In [39]:
print(max(performance_data['Rate_gap']))
print(min(performance_data['Rate_gap']))

2.9655021834061133
-5.801557692307693


In [40]:
performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)

bins = [-6, -2, 0, 1, 2, 3]
labels = ['-6-2%', '-2-0%', '0-1%', '1-2%', '2-3%']

performance_data['Rate_incv'] = pd.cut(performance_data['Rate_gap'], bins=bins, labels=labels, right=False, include_lowest=True)

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_58334/3953439698.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)


In [41]:
performance_data['Rate_incv'].value_counts()

Rate_incv
-2-0%    2358346
0-1%     2259218
1-2%      477533
-6-2%     127476
2-3%        8961
Name: count, dtype: int64

In [42]:
# Make a prepayment rate

# Count of loans with positive rate gap
performance_data['Pos_gap'] = performance_data['Rate_gap'].apply(lambda x: 1 if x > 0 else 0)
performance_data['Neg_gap'] = performance_data['Rate_gap'].apply(lambda x: 1 if x < 0 else 0)

pos = performance_data[performance_data['Pos_gap'] == 1].groupby('MONTHLY_REPORTING_PERIOD')['Pos_gap'].sum().reset_index(name='Pos_gap_count')
neg = performance_data[performance_data['Neg_gap'] == 1].groupby('MONTHLY_REPORTING_PERIOD')['Pos_gap'].sum().reset_index(name='Neg_gap_count')

# If Zero balance code is 1 (prepaid or matured) and the date the balance became zero is before the maturity, the loan is fully prepaid.
condition1 = (performance_data['ZERO_BALANCE_CODE'] ==  1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] < performance_data['MATURITY_DATE'])
prepaynum = performance_data[condition1].groupby('MONTHLY_REPORTING_PERIOD')['Prepayment_status'].sum().reset_index(name='Prepayment_Count')

# THis is a condition for default
condition2 = performance_data['ZERO_BALANCE_CODE'].isin([9, 96, 3, 2])
defaultnum = performance_data[condition2].groupby('MONTHLY_REPORTING_PERIOD')['Default_status'].sum().reset_index(name='Default_Count')


# number of total active loans
totalnum = performance_data.groupby('MONTHLY_REPORTING_PERIOD')['LOAN_SEQUENCE_NUMBER'].size().reset_index(name='Loan_count')

# average rate gap, average loan size, average loan term, average credit score, average loan age, average LTV.
averages = performance_data.groupby('MONTHLY_REPORTING_PERIOD').agg({
    'Rate_gap': 'mean',
    'ORIGINAL_UPB': 'mean',
    'ORIGINAL_LOAN_TERM': 'mean',
    'CREDIT_SCORE': 'mean',
    'ORIGINAL_LOAN_TERM': 'mean',
    'ORIGINAL_LOAN-TO-VALUE': 'mean',
    'ORIGINAL_DEBT-TO-INCOME': 'mean',
    'LOAN_AGE': 'mean'
}).reset_index()

# Merge the results
totalnum = pd.merge(totalnum, averages, on='MONTHLY_REPORTING_PERIOD', how='outer')

final_data = pd.merge(totalnum, defaultnum, on='MONTHLY_REPORTING_PERIOD', how='outer')

# Merge the result with prepaynum
final_data = pd.merge(final_data, prepaynum, on='MONTHLY_REPORTING_PERIOD', how='outer')

final_data = pd.merge(final_data, pos, on='MONTHLY_REPORTING_PERIOD', how='outer')
final_data = pd.merge(final_data, neg, on='MONTHLY_REPORTING_PERIOD', how='outer')

# Fill missing values with 0
final_data.fillna(0, inplace=True)

final_data['Default_rate'] = 100*final_data['Default_Count'] / final_data['Loan_count']
final_data['Prepay_rate'] = 100*final_data['Prepayment_Count'] / final_data['Loan_count']
final_data['Pos_frac'] = 100*final_data['Pos_gap_count'] / final_data['Loan_count']
final_data['Neg_frac'] = 100*final_data['Neg_gap_count'] / final_data['Loan_count']

In [43]:
performance_data.to_csv('LOAN_surv.csv', index=False)
final_data.to_csv('pp_df_rate.csv', index = False)